<a href="https://colab.research.google.com/github/castillosebastian/NLU_legal_domain/blob/master/NLU_Semantica_Vectorial_en_Textos_Legales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semantica Vectorial en Textos Legales: Exploración